In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline

In [2]:
path_client_info = "https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_demo.txt"
path_exp_web_data1 = "https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_1.txt"
path_exp_web_data2 = "https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_2.txt"
path_test_grouping = "https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_experiment_clients.txt"

In [3]:
df_client_info = pd.read_csv(path_client_info)

In [4]:
#check for null values
df_client_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70609 entries, 0 to 70608
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   client_id         70609 non-null  int64  
 1   clnt_tenure_yr    70595 non-null  float64
 2   clnt_tenure_mnth  70595 non-null  float64
 3   clnt_age          70594 non-null  float64
 4   gendr             70595 non-null  object 
 5   num_accts         70595 non-null  float64
 6   bal               70595 non-null  float64
 7   calls_6_mnth      70595 non-null  float64
 8   logons_6_mnth     70595 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 4.8+ MB


In [5]:
#dropping nan dominated rows
df_client_info.dropna(thresh=len(df_client_info.columns) - 3,inplace=True)
df_client_info.info()
#14 rows / clients dropped

<class 'pandas.core.frame.DataFrame'>
Index: 70595 entries, 0 to 70608
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   client_id         70595 non-null  int64  
 1   clnt_tenure_yr    70595 non-null  float64
 2   clnt_tenure_mnth  70595 non-null  float64
 3   clnt_age          70594 non-null  float64
 4   gendr             70595 non-null  object 
 5   num_accts         70595 non-null  float64
 6   bal               70595 non-null  float64
 7   calls_6_mnth      70595 non-null  float64
 8   logons_6_mnth     70595 non-null  float64
dtypes: float64(7), int64(1), object(1)
memory usage: 5.4+ MB


In [6]:
#load second table, the experiment's web data table 1
df_exp_web_data1 = pd.read_csv(path_exp_web_data1)
df_exp_web_data1
#343141 rows

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04
...,...,...,...,...,...
343136,2443347,465784886_73090545671,136329900_10529659391_316129,confirm,2017-03-31 15:15:46
343137,2443347,465784886_73090545671,136329900_10529659391_316129,step_3,2017-03-31 15:14:53
343138,2443347,465784886_73090545671,136329900_10529659391_316129,step_2,2017-03-31 15:12:08
343139,2443347,465784886_73090545671,136329900_10529659391_316129,step_1,2017-03-31 15:11:37


In [ ]:
#load third table, the experiment's web data table 2
df_exp_web_data2 = pd.read_csv(path_exp_web_data2)
df_exp_web_data2
#412264 rows

In [ ]:
#merge the two experiment tables via concat:
df_exp_web_data = pd.concat([df_exp_web_data1, df_exp_web_data2], axis=0)
df_exp_web_data
#755405 rows

In [ ]:
df_exp_web_data['client_id'].nunique()
#120k clients, why?

In [ ]:
#check fourth table of test grouping info per client
df_ab_test_grouping = pd.read_csv(path_test_grouping)
df_ab_test_grouping
#70609 rows -> matches with number of client_ids in df_client_info

In [ ]:
#check for NaN values:
df_ab_test_grouping.isna().sum()
# 0 NaNs for client_id but 20109 for Variation -> 20k of 70k clients did not
#participate in the test?

In [ ]:
#Finally cut out the customers that are not participating in the A/B trials
participation_df = df_ab_test_grouping[~df_ab_test_grouping['Variation'].isnull()]
participation_df
#50500 clients left

In [ ]:
#retrieve client_ids of the 50.500 participating clients and check if apparent in merged web experiment table, if yes
#filter to only keep these for our experiment analysis
participation_filter = participation_df["client_id"].values
df_exp_web_data_filtered = df_exp_web_data[df_exp_web_data['client_id'].isin(participation_filter)]
df_exp_web_data_filtered
#321309 rows left

In [ ]:
#check number of remaining clients
df_exp_web_data_filtered["client_id"].nunique()
#50.500; initially 120157 clients in web_exp data -> 69657 completely new clients within
#the web_exp data but why? are there client_ids in the web_exp that are not in the two other tables?
# the additional 49548 clients will be kicked out by the analysis of the test result anyways, 
#as they dont have a test grouping

In [ ]:
#Further EDA & Cleaning: check relationship between client_id, visitor_id and visit_id in web_exp_ table:
df_exp_web_data_filtered

In [ ]:
#merge info about grouping (test vs. control) of clients into web experiment data
df_web_exp = df_exp_web_data_filtered.merge(participation_df, on='client_id', how='left')
df_web_exp['Variation'].value_counts()
#Test       177847
#Control    143462

In [ ]:
df_web_exp.info() #321309 entries
df_web_exp['visitor_id'].nunique() #56011 -> ~500 more than client_id

In [ ]:
#check for visitor_ids with multiple client ids:
#group by 'visitor_id'
grouped = df_web_exp.groupby('visitor_id')

#find groups where 'Variation' has more than one unique value
ambiguous_visitors = grouped.filter(lambda x: x['Variation'].nunique() > 1)

#get unique visitor IDs from confusing groups
ambiguous_visitor_ids = ambiguous_visitors['visitor_id'].unique()

#output the result
print(f"Visitor_id groups with more than one unique 'Variation' value: {len(ambiguous_visitor_ids)}")
#print(f"List of ambiguous visitor group IDs: {ambiguous_visitor_ids}")
#185 ambiguous visitor_ids

In [ ]:
#example check
df_web_exp[df_web_exp['visitor_id'] == '454881699_42713236223']

In [ ]:
#as they have different variations but the same visitor_id (same device?) and even the same
#visit_id (sessions?) in some cases we have to get rid of these 185 visitor_ids
#from 185 visitor_ids that have more than one variation take the client ids into a drop list
client_ids_to_drop = []
for visitor_id in ambiguous_visitor_ids:
    for i in range(len(df_web_exp[df_web_exp['visitor_id'] == f'{visitor_id}']['client_id'].unique())):
        client_ids_to_drop.append(df_web_exp[df_web_exp['visitor_id'] == f'{visitor_id}']['client_id'].unique()[i])

In [ ]:
# Drop rows where client_id is in client_ids_to_drop
df_web_exp = df_web_exp[~df_web_exp['client_id'].isin(client_ids_to_drop)]
#df_web_exp.info() #318872 entries, before 321309 ->3k process steps / timestamps dropped
df_web_exp['visitor_id'].nunique() #55788, before 56011 -> 230 clients dropped

In [ ]:
## CSV Creation

df_web_exp.to_csv('data_web_exp.csv',index=False)
df_client_info.to_csv('data_client_info.csv',index=False)
df_ab_test_grouping.to_csv('data_ab_test_grouping.csv',index=False)